In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import json

In [51]:
DRIVER_PATH = "../driver/chromedriver"

collegedata_general = {}

importance = ["Very Important", "Important", "Considered", "Not Considered"]

## Scraping General Function

In [82]:
"""
scrape function:
1. access each college page using college_links.json
2. wait for 2 seconds
3. scrape each page (find the button on the navbar and click)
    - wait for 1 second after click
4. wrap up in json format and close
"""

def get_general_college_data(college_name, college_link, sleep=0.5):
    driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    cur_college_data = {}

    # Admission Page

    driver.get(college_link + "/admission")
    time.sleep(sleep)
    admission_body = WebDriverWait(driver, sleep).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    print(get_admission(admission_body, 0.5))

    # Financials Page

    driver.get(college_link + "/money-matters")
    time.sleep(sleep)
    financials_body = WebDriverWait(driver, sleep).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    # Academics Page

    driver.get(college_link + "/academics")
    time.sleep(sleep)
    academics_body = WebDriverWait(driver, sleep).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    # Campus Life page

    driver.get(college_link + "/campus-life")
    time.sleep(sleep)
    campus_life_body = WebDriverWait(driver, sleep).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    # Students Page

    driver.get(college_link + "/students")
    time.sleep(sleep)
    students_body = WebDriverWait(driver, sleep).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    # quit
    driver.quit()
    collegedata_general[college_name] = cur_college_data


## Admission page function

In [98]:
def get_admission(admission_body, sleep=0.5):
    admission_info = {}

    all_info = admission_body.find_elements(By.CSS_SELECTOR, value=".CollegeProfileContent_expandableChildren__3bIKY")

    # Freshman admission requirements

    admission_info["high_school_req_rec_data"] = {}

    high_school_req_rec = admission_body.find_element(By.CSS_SELECTOR, value=".Table_unlined__29Tmx > tbody")
    high_school_req_rec_table = high_school_req_rec.find_elements_by_css_selector("tr")
    for unit in high_school_req_rec_table:
        elements = unit.find_elements_by_css_selector("td")
        admission_info["high_school_req_rec_data"][elements[0].text] = {
            "required": int(elements[1].text) if elements[1].text != '' else None,
            "recommended": int(elements[2].text) if elements[2].text != '' else None
        }

    time.sleep(sleep)

    # Applying for admission

    apply_adm = all_info[1]
    adm_contents = apply_adm.find_elements(By.CSS_SELECTOR, value=".CollegeProfileContent_content__1hJCl > .CollegeProfileContent_content__1hJCl")

    for adm_content in adm_contents:
        try:
            title = adm_content.find_element(By.CLASS_NAME, value="TitleValue_title__2-afK").text
            info = adm_content.find_element(By.CLASS_NAME, value="TitleValue_value__1JT0d").text
            admission_info[title] = info
        except:
            continue

    # Selection of students

    admission_info["selection_of_students"] = {}
    
    selection_of_students_table = all_info[2].find_element(By.TAG_NAME, value="tbody")
    metrics = selection_of_students_table.find_elements(By.TAG_NAME, value="tr")

    for metric in metrics:
        elements = metric.find_elements(By.TAG_NAME, value="td")
        index = 0
        metric_name = ""
        for element in elements:
            if index == 0:
                metric_name = element.text
            else:
                if element.text == "X":
                    admission_info["selection_of_students"][metric_name] = importance[index - 1]
                    break
            index += 1
    
    # Profile of Fall Application

    fall_app_prof = all_info[3]
    prof_contents = fall_app_prof.find_elements(By.CSS_SELECTOR, value=".CollegeProfileContent_content__1hJCl > .CollegeProfileContent_content__1hJCl")

    for prof_content in prof_contents:
        try:
            text_title = prof_content.find_element(By.CLASS_NAME, value="TitleValue_title__2-afK").text
            text_info = prof_content.find_element(By.CLASS_NAME, value="TitleValue_value__1JT0d").text
            admission_info[text_title] = text_info
        except:
            pass

        try:
            histogram = prof_content.find_element(By.CLASS_NAME, value="Histogram_container__12lhQ")
        except:
            continue

        try:
            histogram_title = prof_content.find_element(By.CLASS_NAME, value="CollegeProfileContent_barGraphTitle__1uzxw").text
            print(histogram_title)
            histogram_title_list = histogram_title.split(": ")
            test_title = ""
            overall_intro = ""
            if len(histogram_title_list) == 1:
                test_title = "ACT"
                overall_intro = histogram_title_list[0]
            else:
                [test_title, overall_intro] = histogram_title_list
            admission_info[test_title] = {
                "overall": overall_intro,
                "data": histogram_scrape(prof_content)
            }
            # print(admission_info)
        except:
            admission_info["GPA"] = histogram_scrape(prof_content)

    return admission_info

# histogram process function
def histogram_scrape(prof_content):
    histogram_data = {}
    histogram = prof_content.find_element(By.CLASS_NAME, value="Histogram_container__12lhQ")
    histogram_attr = histogram.find_elements(By.CLASS_NAME, value="Histogram_label__1j_Ku")
    histogram_val = histogram.find_elements(By.CLASS_NAME, value="Histogram_value__3cCAB")
    for each_attr, each_val in zip(histogram_attr, histogram_val):
        histogram_data[each_attr.text] = each_val.text
    return histogram_data

## Test

In [99]:
get_general_college_data("Abilene Christian University", "https://waf.collegedata.com/college-search/Abilene-Christian-University")

/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # This is added back by InteractiveShellApp.init_path()
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  # This is added back by InteractiveShellApp.init_path()
/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  del sys.path[0]


SAT Math: 554 average , 500-600 range of middle 50%
SAT EBRW: 568 average , 510-620 range of middle 50%
24 average , 21-27 range of middle 50%
ACT Math: 23
ACT Eng: 24
{'high_school_req_rec_data': {'English': {'required': None, 'recommended': 4}, 'Mathematics': {'required': None, 'recommended': 3}, 'Science': {'required': None, 'recommended': 3}, 'Foreign Language': {'required': None, 'recommended': 2}, 'History': {'required': None, 'recommended': 1}}, 'Address': '1600 Campus Court', 'Phone': '(800) 460-6228', 'Regular Admission Deadline': 'Rolling', 'Application Fee': '$50', 'Application Fee Waiver': 'Available', 'Regular Admission Notification': 'Rolling, notification begins March 15', 'Accept Offer of Admission': 'May 1', 'Waiting List Used': 'No', 'Defer Admission': 'Student cannot defer admission', 'Transfer Admission': 'Transfer applications accepted', 'Early Decision Offered': 'No', 'Early Action Offered': 'Yes', 'Early Action Deadline': 'November 1', 'Early Action Notification'